## MeanShift

In [37]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn import preprocessing
import numpy as np
import pandas as pd
import time


def del_repeate(data):
    dic = {}
    del_list = []
    for i in range(data.shape[0]):
        if int(data[i,0]) not in dic:
            dic[int(data[i,0])] = 1
        else:
            del_list.append(i)
            
    data = np.delete(data,del_list,axis=0)
    print("delete repeat.....",data.shape)
    return data

def view_count(data):
    dic = {}
    for i in range(data.shape[0]):
        if int(data[i,0]) not in dic:
            dic[int(data[i,0])] = 1
        else:
            dic[int(data[i,0])] = dic[int(data[i,0])] + 1
    
    x = np.zeros((data.shape[0],1), dtype=int)
    for i in range(data.shape[0]):
        x[i] = dic[int(data[i,0])]
        
    data = np.concatenate((data,x),axis=1)
    print("n-1 days viewcount.....",data.shape)
    return data

def Data_compare(data,window):
    view_count = 0
    request_num = 0
    start_time = -1
    timestamp = 0
    end_time = 0
    idx = []
    for i in range(data.shape[0]):
        timestamp = int(data[i,0][3:10])
        if start_time == -1:
            start_time = timestamp + (window-1) * 86400
            end_time = start_time + window * 86400
        if end_time < timestamp:

            break
        if (timestamp >= start_time and timestamp <= end_time):
            idx.append(i)
        request_num += 1
    print(idx[0],'min',idx[-1],'max')
    #print(idx)
    #print("req.num",request_num)
    #data = data[:request_num+1]
    #2018/09/21
    #data = np.delete(data,[0,1,2,3,5,10],axis=1)
    #data = np.delete(data,0,axis=1)

    #print("%d days shape "% window,data.shape)
    print(data)
    return idx

def CacheHit(data):
    hit = 0
    for video_id in data[:,0]:
        if video_id in cache_content:
            hit += 1
    return hit
        

def UpdateCache(cache_size,url_data,x_label,most_freq):
    # == 最多群開啟此行
    global cache_content
    cache_content.clear()
    print("most cluster:",np.argmax(most_freq,axis=0))
    while(len(cache_content) < cache_size):
        most_cluster = np.argmax(most_freq,axis=0)
        #print("most_cluster",most_cluster)
        idex = np.where(x_label != most_cluster)[0]
        for i in idex:
            if url_data[i,0] not in cache_content:
                if len(cache_content) < cache_size:
                    cache_content.add(url_data[i,0])
                else:
                    break
            #else:
                #most_freq[most_cluster] = most_freq[most_cluster] - 1
                    
        # != 最多群開啟下面
        
        idex = np.where(x_label == most_cluster)[0]
        for i in idex:
            if url_data[i,0] not in cache_content:
                if len(cache_content) < cache_size:
                    cache_content.add(url_data[i,0])
                    most_freq[most_cluster] = most_freq[most_cluster] - 1
                else:
                    break
            else:
                most_freq[most_cluster] = most_freq[most_cluster] - 1

    #print("length:",len(cache_content))
                

def BuildModel(train):
    train = np.delete(train,[0],axis=1)
    global cluster,X_pred
    cluster = []
    X = train.astype(np.float32)
    #test = test.astype(np.float32)
    sc = StandardScaler()
    sc.fit(X)
    X = sc.transform(X)
    #test = sc.transform(test)
    pca = PCA(n_components=2)
    pca.fit(X)
    X = pca.transform(X)
    #test = pca.transform(test)
    
    #bandwidth = estimate_bandwidth(X , quantile=0.3, n_samples=X.shape[0])
    ms = MeanShift(bin_seeding=True,n_jobs=-1)
    ms.fit(X)
    X_pred = ms.predict(X)
    labels = ms.labels_
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)

    #print("y.shape",y.shape)
    
    #print(n_clusters_,end="")
    cluster = ms.cluster_centers_[:,1]
    #print("\nCenter cluster",cluster)
    print("model finished!!")
    return X_pred
    
#all_data = np.concatenate((data_2[1:y.shape[0]+1,:],y.reshape(-1,1)),axis=1)
if __name__ == '__main__':  
    global cluster,y,cache_content
    cache_content = set()
    cluster = []
    #data_1 = np.loadtxt('./training.txt',dtype='U16',delimiter=' ')
    #2018/09/20
    data_1 = np.loadtxt('./data_for_meanshift_birch.csv',dtype='U16',delimiter=',')
    
    test_idx = Data_compare(data_1,7)
    #data_new = np.loadtxt('./train_new.txt',dtype='U16',delimiter=' ')
    data_1 = np.delete(data_1,0,axis=1)
    X_train = data_1[:test_idx[0]]
    
    #X_train = view_count(X_train)
    
    X_train = del_repeate(X_train)


    X_test = data_1[test_idx[0]:test_idx[-1]+1]
    #data_new = Data_preprocess(all_data)

    hits = [0]
    hit_rate = [0]
    hit_idx = 1

    index = X_test.shape[0]
    
    
    start_ts = time.time()
    #X_pred = BuildModel(X_train) #如果跑第2次就註解掉
    end_ts = time.time()
    
    print("cost time: \n",end_ts-start_ts)
    for j in range(5000,50000+1,5000):
        cache_content.clear()
        print("request\tvideo\tCluster\thit\tC="+str(j))

        #hits.append(CacheHit(X_test))
        
        #print("X_pred",X_pred)
        most_item = np.bincount(X_pred) # 有最多元素的cluster
        #print("most_item",most_item)
        #for i in range(y.shape[0]):
            #y[i] = cluster[int(y[i])]
        #big = np.amax(cluster)
        #ls = sorted(cluster)

        UpdateCache(j,X_train,X_pred,most_item)
        print("len",len(cache_content))
        #BuildModel(X_new,X_test) 

        hits.append(CacheHit(X_test))
        print("hits",hits)
        #print(cache_content)
        hit_rate.append(hits[hit_idx]/index*100)
        hit_idx += 1
        print("\thitrate",hit_rate)


        print("===============================")


            


110415 min 248457 max
[['1201639761.78066' '0' '24' ... '24' '8' '1']
 ['1201639767.58091' '1' '25' ... '30' '87' '1']
 ['1201639770.34167' '2' '23' ... '5' '4' '1']
 ...
 ['1202849965.87344' '80016' '17' ... '151' '997' '13']
 ['1202849966.01782' '121049' '17' ... '18' '314' '1']
 ['1202849966.93319' '121050' '22' ... '0' '2' '1']]
delete repeat..... (57353, 7)
cost time: 
 0.0
request	video	Cluster	hit	C=5000
most cluster: 0
len 5000
hits [0, 10917]
	hitrate [0, 7.908405351955549]
request	video	Cluster	hit	C=10000
most cluster: 0
len 10000
hits [0, 10917, 17019]
	hitrate [0, 7.908405351955549, 12.32876712328767]
request	video	Cluster	hit	C=15000
most cluster: 0
len 15000
hits [0, 10917, 17019, 20935]
	hitrate [0, 7.908405351955549, 12.32876712328767, 15.16556435313634]
request	video	Cluster	hit	C=20000
most cluster: 0
len 20000
hits [0, 10917, 17019, 20935, 24899]
	hitrate [0, 7.908405351955549, 12.32876712328767, 15.16556435313634, 18.037133356997455]
request	video	Cluster	hit	C=250

## Birch

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AffinityPropagation,SpectralClustering,DBSCAN,AgglomerativeClustering,Birch
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing
import numpy as np
import pandas as pd
import time

def del_repeate(data):
    dic = {}
    del_list = []
    for i in range(data.shape[0]):
        if int(data[i,0]) not in dic:
            dic[int(data[i,0])] = 1
        else:
            del_list.append(i)
            
    data = np.delete(data,del_list,axis=0)
    print("delete repeat.....",data.shape)
    return data


def Data_compare(data,window):
    view_count = 0
    request_num = 0
    start_time = -1
    timestamp = 0
    end_time = 0
    idx = []
    for i in range(data.shape[0]):
        timestamp = int(data[i,0][3:10])
        if start_time == -1:
            start_time = timestamp + (window-1) * 86400
            end_time = start_time + window * 86400
        if end_time < timestamp:

            break
        if (timestamp >= start_time and timestamp <= end_time):
            idx.append(i)
        request_num += 1
    print(idx[0],'min',idx[-1],'max')
    #print(idx)
    #print("req.num",request_num)
    #data = data[:request_num+1]
    #2018/09/21
    #data = np.delete(data,[0,1,2,3,5,10],axis=1)
    #data = np.delete(data,0,axis=1)

    #print("%d days shape "% window,data.shape)
    print(data)
    return idx

def CacheHit(data):
    hit = 0
    for video_id in data[:,0]:
        if video_id in cache_content:
            hit += 1
    return hit


def UpdateCache(cache_size,url_data,x_label,most_freq):
    # == 最多群開啟此行
    global cache_content
    cache_content.clear()
    print("most cluster:",np.argmax(most_freq,axis=0))
    while(len(cache_content) < cache_size):
        most_cluster = np.argmax(most_freq,axis=0)
        #print("most_cluster",most_cluster)
        idex = np.where(x_label != most_cluster)[0]
        for i in idex:
            if url_data[i,0] not in cache_content:
                if len(cache_content) < cache_size:
                    cache_content.add(url_data[i,0])
                else:
                    break
            #else:
                #most_freq[most_cluster] = most_freq[most_cluster] - 1
                    
        # != 最多群開啟下面
        
        idex = np.where(x_label == most_cluster)[0]
        for i in idex:
            if url_data[i,0] not in cache_content:
                if len(cache_content) < cache_size:
                    cache_content.add(url_data[i,0])
                    most_freq[most_cluster] = most_freq[most_cluster] - 1
                else:
                    break
            else:
                most_freq[most_cluster] = most_freq[most_cluster] - 1

    #print("length:",len(cache_content))
                

def BuildModel(train):
    train = np.delete(train,[0],axis=1)
    #test = np.delete(test,[0],axis=1)
    cluster = []
    X = train.astype(np.float32)
    #test = test.astype(np.float32)
    sc = StandardScaler()
    sc.fit(X)
    X = sc.transform(X)
    #test = sc.transform(test)
    pca = PCA(n_components=4)
    pca.fit(X)
    X = pca.transform(X)
    #test = pca.transform(test)

    af = Birch(n_clusters=6)
    #af.fit(X)
    X_pred = af.fit_predict(X)
    #center = af.subcluster_centers_
    #print("####################",center.shape)
    labels = af.labels_
    print(labels)
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)
    print(X_pred)
    #y = af.fit_predict(test)
    
    print("model finished!!")
    return X_pred

    
if __name__ == '__main__':  
    global cluster,y,cache_content
    cache_content = set()
    cluster = []

    data_1 = np.loadtxt('./data_for_meanshift_birch.csv',dtype='U16',delimiter=',')
    
    test_idx = Data_compare(data_1,7)
    data_1 = np.delete(data_1,0,axis=1)
    X_train = data_1[:test_idx[0]]
    #X_train = data_new[:train_samples,]

    X_train = del_repeate(X_train)
    
    print(X_train[0])
    #X_update = data_new[-69616:-11051]
    X_test = data_1[test_idx[0]:test_idx[-1]+1,[0]]
    print("train.shape",X_train.shape)
    #print(X_update.shape)
    print("test.shape",X_test.shape)

    hits = [0]
    hit_rate = [0]
    hit_idx = 1

    index = X_test.shape[0]
    
    start_ts = time.time()
    X_pred = BuildModel(X_train) #如果跑第2次就註解掉
    end_ts = time.time()
    
    print("cost time: \n",end_ts-start_ts)
    
    for j in range(500,3000+1,500):
        cache_content.clear()
        print("request\tvideo\tCluster\thit\tC="+str(j))

        most_item = np.bincount(X_pred)
        
        UpdateCache(j,X_train,X_pred,most_item)
        print("len",len(cache_content))


        hits.append(CacheHit(X_test))
        print("hits",hits)
        hit_rate.append(hits[hit_idx]/index*100)
        hit_idx += 1
        print("hitrate",hit_rate)


        print("===============================")


110415 min 248457 max
[['1201639761.78066' '0' '24' ..., '24' '8' '1']
 ['1201639767.58091' '1' '25' ..., '30' '87' '1']
 ['1201639770.34167' '2' '23' ..., '5' '4' '1']
 ..., 
 ['1202849965.87344' '80016' '17' ..., '151' '997' '13']
 ['1202849966.01782' '121049' '17' ..., '18' '314' '1']
 ['1202849966.93319' '121050' '22' ..., '0' '2' '1']]
delete repeat..... (57353, 7)
['0' '24' '12895' '4' '24' '8' '1']
train.shape (57353, 7)
test.shape (138043, 1)
[3 3 3 ..., 3 3 3]
[3 3 3 ..., 3 3 3]
model finished!!
cost time: 
 4.014533042907715
request	video	Cluster	hit	C=500
most cluster: 3
len 500
hits [0, 3222]
hitrate [0, 2.3340553305853975]
request	video	Cluster	hit	C=1000
most cluster: 3
len 1000
hits [0, 3222, 4085]
hitrate [0, 2.3340553305853975, 2.959222850850822]
request	video	Cluster	hit	C=1500
most cluster: 3
len 1500
hits [0, 3222, 4085, 5023]
hitrate [0, 2.3340553305853975, 2.959222850850822, 3.6387212680106926]
request	video	Cluster	hit	C=2000
most cluster: 3
len 2000
hits [0, 322